# Извлечение данных из коллекции новостных текстов

Данные расположены  [в файле data/news.tar.gz](data/news.tar.gz). С некоторых новостных сайтов был загружен архив новостей а  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`
    

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

        sport <tab> Сборная Канады по хоккею разгромила чехов <tab> Сборная Канады по хоккею крупно об...

С помощью [Yargy](https://github.com/natasha/yargy) или [Томита-парсера](https://github.com/yandex/tomita-parser) извлеките данные, которые можно описать структурой вида:


```python
@dataclass
class Entry:
    name: str
    birth_date: Optional[str]
    birth_place: Optional[str]
```

In [1]:
import gzip

from dataclasses import dataclass
from typing import Iterator

@dataclass
class Text:
    label: str
    title: str
    text: str
    
file_path = "D://Nlp'23//nlp-2023//data//news.txt.gz"

def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))

texts = list(read_texts(file_path))


In [2]:
from yargy import Parser, rule, and_, not_, or_
from yargy.predicates import dictionary,  gte, lte, is_capitalized
from yargy.pipelines import  morph_pipeline, pipeline
from yargy.interpretation import fact
from yargy.predicates import gram
from yargy.relations import gnc_relation

Name = fact(
    'Name',
    ['first', 'last'],
)

Date = fact(
    'Date',
    ['year', 'month', 'day']
)

Birth_place = fact(
    'Birth_place',
    ['name']
)

Entry = fact(
    'Entry',
    ['full_name', 'birth_date', 'birth_place']
)


In [3]:
LAST = and_(
    gram('Surn'),
    not_(gram('Abbr')),
)

FIRST = and_(
    gram('Name'),
    not_(gram('Abbr')),
)

gnc = gnc_relation()

NAME = rule( 
    or_(
    
    rule(
        FIRST.interpretation(Name.first).match(gnc),
        LAST.interpretation(Name.last).match(gnc)
        ),
    rule(
        LAST.interpretation(Name.last).match(gnc),
        FIRST.interpretation(Name.first).match(gnc)
        )
       )
    ).interpretation(
    Name
    )

In [4]:
BORN_VERB = morph_pipeline([                            
                             'появился в',                              
                             'появился на свет в',     
                             'родился в',                            
                             'появился',                              
                             'появился на свет',     
                             'родился',
                             'был рожден'      
                            ])

BIRTH_PlACE_CITY = rule(
    gram("NOUN").optional().repeatable(),
    and_(
        is_capitalized(),
        gram("NOUN")
    )
).interpretation(Birth_place.name) 

BIRTH_PLACE = rule(
    BIRTH_PlACE_CITY.interpretation(Birth_place.name)
    ).interpretation(Birth_place)


In [5]:

BIRTH_TAG = morph_pipeline([                            
                             'в'     
                            ])

MONTHS = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'май': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12
}

DATE_SEP = pipeline(['.', ':', '/'])

MONTH_NAME = dictionary(
    MONTHS
).interpretation(
    Date.month.normalized().custom(MONTHS.get)
)

DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Date.day.custom(int)
)

YEAR = and_(
    gte(1900),
    lte(2100)
).interpretation(
    Date.year.custom(int)
)

DATE_WITH_MONTH_IN_NUM = rule(
    DAY,
    DATE_SEP.optional (),

    and_( gte(1),
          lte(12)
        ).interpretation(
     Date.month.custom(int)
    ), 

    DATE_SEP.optional(),
    YEAR.optional()
)

DATE_WITH_MONTH_IN_WORD = rule(
    DAY,
    DATE_SEP.optional(),
    MONTH_NAME.optional(), 
    DATE_SEP.optional(),
    YEAR.optional()
)

# разные версии представления даты: месяц словом и через раздилитель (., / или :)

DATE = rule(
    or_(
        DATE_WITH_MONTH_IN_WORD, 
        DATE_WITH_MONTH_IN_NUM
       )
).interpretation(Date)


In [6]:
from tqdm import tqdm 

#разные комбинации информации о человеке

RULE = or_ (

    rule(    
        NAME.interpretation(Entry.full_name),
        BORN_VERB,
        BIRTH_TAG.optional(),
        BIRTH_PLACE.interpretation(Entry.birth_place).optional(),
        BIRTH_TAG.optional(),
        DATE.interpretation(Entry.birth_date).optional()
    ),


    rule(
        NAME.interpretation(Entry.full_name),
        BORN_VERB,
        BIRTH_TAG.optional(),
        DATE.interpretation(Entry.birth_date).optional(),
        BIRTH_TAG.optional(),
        BIRTH_PLACE.interpretation(Entry.birth_place).optional()
    ),

    rule(
        NAME.interpretation(Entry.full_name),
        BIRTH_TAG.optional(),
        DATE.interpretation(Entry.birth_date).optional(),
        BORN_VERB,
        BIRTH_TAG.optional(),
        BIRTH_PLACE.interpretation(Entry.birth_place).optional()
    ),

        rule(
        NAME.interpretation(Entry.full_name),
        BIRTH_TAG.optional(),
        BIRTH_PLACE.interpretation(Entry.birth_place).optional(),
        BORN_VERB,
        BIRTH_TAG.optional(),
        DATE.interpretation(Entry.birth_date).optional()
    ),

    rule( 
        NAME.interpretation(Entry.full_name)
        )
    
    ).interpretation(
    Entry
)

In [7]:
parser = Parser(RULE)

for text in tqdm(texts, disable=True):
    try:
        for match in parser.findall(text.text):
         print(match.fact)
    except:
       pass

Entry(full_name=Name(first='Матс', last='Сундин'), birth_date=None, birth_place=None)
Entry(full_name=Name(first='Владимир', last='Филиппов'), birth_date=None, birth_place=None)
Entry(full_name=Name(first='Ильгизу', last='Фахриеву'), birth_date=None, birth_place=None)
Entry(full_name=Name(first='Михаилом', last='Прохоровым'), birth_date=None, birth_place=None)
Entry(full_name=Name(first='Бена', last='Николсона'), birth_date=None, birth_place=None)
Entry(full_name=Name(first='Анри', last='Матисса'), birth_date=None, birth_place=None)
Entry(full_name=Name(first='Пабло', last='Пикассо'), birth_date=None, birth_place=None)
Entry(full_name=Name(first='Жоржа', last='Брака'), birth_date=None, birth_place=None)
Entry(full_name=Name(first='Владимиром', last='Путиным'), birth_date=None, birth_place=None)
Entry(full_name=Name(first='Леонида', last='Брежнева'), birth_date=None, birth_place=None)
Entry(full_name=Name(first='Юрий', last='Андропов'), birth_date=None, birth_place=None)
Entry(full_name